<a href="https://colab.research.google.com/github/JuanPabl07DP/Cardiovascular_FNN/blob/main/Cardiovascular_FNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sulianova/cardiovascular-disease-dataset")

print("Path to dataset files:", path)

100%|██████████| 742k/742k [00:00<00:00, 867kB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/sulianova/cardiovascular-disease-dataset/versions/1


In [2]:
import pandas as pd

df = pd.read_csv(f"{path}/cardio_train.csv", sep=';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [3]:
import numpy as np

# Eliminar columnas innecesarias (ID si existe)
df = df.drop(columns=['id'], errors='ignore')

# Convertir 'gender' a binario (0 = mujer, 1 = hombre)
df['gender'] = df['gender'].map({1: 1, 2: 0})

# Convertir 'age' de días a años
df['age'] = (df['age'] / 365.25).astype(int)

# Convertir 'weight' a enteros
df['weight'] = df['weight'].astype(int)

# Revisar valores extremos
df = df[(df['height'] > 100) & (df['height'] < 250)]
df = df[(df['weight'] > 30) & (df['weight'] < 200)]
df = df[(df['ap_hi'] > 70) & (df['ap_hi'] < 250)]
df = df[(df['ap_lo'] > 40) & (df['ap_lo'] < 200)]

# Normalización
from sklearn.preprocessing import MinMaxScaler

features = df.drop(columns=['cardio'])
labels = df['cardio']

scaler = MinMaxScaler()
X = features.values
y = labels.values

In [4]:
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,50,0,168,62,110,80,1,1,0,0,1,0
1,55,1,156,85,140,90,3,1,0,0,1,1
2,51,1,165,64,130,70,3,1,0,0,0,1
3,48,0,169,82,150,100,1,1,0,0,1,1
4,47,1,156,56,100,60,1,1,0,0,0,0


In [5]:
from sklearn.model_selection import train_test_split

# División
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_dev, y_train, y_dev = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

# Escalado
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_dev_scaled = scaler.transform(X_dev)
X_test_scaled = scaler.transform(X_test)

print(f'Train: {len(X_train)} samples')
print(f'Dev: {len(X_dev)} samples')
print(f'Test: {len(X_test)} samples')

Train: 48119 samples
Dev: 10279 samples
Test: 10306 samples


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(256, input_dim=X_train_scaled.shape[1], kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),
    Dropout(0.4),

    Dense(128, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),
    Dropout(0.3),

    Dense(64, kernel_regularizer=l2(0.001)),
    LeakyReLU(alpha=0.01),

    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='binary_crossentropy',
              metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [8]:
history = model.fit(X_train_scaled, y_train,
                    validation_data=(X_dev_scaled, y_dev),
                    epochs=40,
                    batch_size=128,
                    callbacks=[early_stop, reduce_lr],
                    verbose=1)

Epoch 1/40
376/376 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.7034 - loss: 0.8107 - val_accuracy: 0.7291 - val_loss: 0.6727 - learning_rate: 5.0000e-04
Epoch 2/40
376/376 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7272 - loss: 0.6585 - val_accuracy: 0.7266 - val_loss: 0.6146 - learning_rate: 5.0000e-04
Epoch 3/40
376/376 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7299 - loss: 0.6068 - val_accuracy: 0.7281 - val_loss: 0.5865 - learning_rate: 5.0000e-04
Epoch 4/40
376/376 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7309 - loss: 0.5839 - val_accuracy: 0.7285 - val_loss: 0.5734 - learning_rate: 5.0000e-04
Epoch 5/40
376/376 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7348 - loss: 0.5691 - val_accuracy: 0.7291 - val_loss: 0.5653 - learning_rate: 5.0000e-04
Epoch 6/40
376/376 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7329 - loss: 0.5644 - val_accuracy: 0.7322 - val_loss: 0.5606 - learning_rate: 5.0000e-04
Epoch 7/40
376/376 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 

In [9]:
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print(f"Test Accuracy: {test_acc:.4f}, Test Loss: {test_loss:.4f}")

323/323 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7317 - loss: 0.5476
Test Accuracy: 0.7301, Test Loss: 0.5503


In [10]:
model.save("modelo_cardio_hibrido_fnn.h5")